### Практическое задание к уроку 6
**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# os.environ['MKL_NUM_THREADS'] = '1'

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
import src.features as ft
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings('ignore')

C:\Users\AVasilev\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 10 

# Process features dataset

In [4]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [5]:
VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [6]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [7]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [8]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [9]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [10]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


# Prefilter items

In [11]:
# Определю количество наиболее популярных товаров, которые составляют 90% продаж
items_by_poprularity = data_train_matcher.groupby(by='item_id')['basket_id'].nunique(). \
    reset_index().sort_values(by='basket_id', ascending=False)
items_by_poprularity.rename(columns={'basket_id': 'n_purchases'}, inplace=True)
items_by_poprularity.head()

,item_id,n_purchases
34192,1082185,24318
54389,6534178,16233
28450,1029743,11661
24657,995242,10226
36790,1106523,8011


In [12]:
purchses_sum = items_by_poprularity.n_purchases.sum()
top_90_percent_items_list = []
purchses_commul_sum = 0
for item, n_purchases in zip(items_by_poprularity.item_id, items_by_poprularity.n_purchases):
    purchses_commul_sum += n_purchases
    if (purchses_commul_sum / purchses_sum) < 0.9:
        top_90_percent_items_list.append(item)
    else:
        break

n_popular = len(top_90_percent_items_list)
n_popular

18714

In [13]:
# Оставляю в датасете только 18714 товаров. Id остальных заменю на 999999
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, 
                                     take_n_popular=n_popular)

n_items_after = data_train_matcher['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 83685 to 18715


# Make cold-start to warm-start

In [14]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 18715
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


# Grid search

In [15]:
# from itertools import product

In [16]:
# weightings_list = ['bm25', 'tfidf']
# model_type_list = ['als', 'bpr']
# user_item_matrix_values = ['binary', 'sales_value', 'purchase_sum']
# own_recommender_type_list = ['item-item', 'cosine', 'tfidf']
# recs_type_list = ['own', 'rec', 'itm', 'usr']

In [17]:
# result_dict = {
#     'weighting': [],
#     'model_type': [],
#     'own_recommender_type': [],
#     'user_item_matrix_values': [],
#     'own_recall': [],
#     'rec_recall': [],
#     'itm_recall': [],
#     'usr_recall': []
# }

In [18]:
# %%time
# for ui_value, weighting, model_type, own_recommender in product(
#             user_item_matrix_values, weightings_list, model_type_list, own_recommender_type_list):
#     base_recommender = MainRecommender(data_train_matcher, weighting=weighting, 
#                                        model_type=model_type, own_recommender_type=own_recommender, 
#                                        user_item_matrix_values=ui_value)
#     result_dict['weighting'].append(weighting)
#     result_dict['model_type'].append(model_type)
#     result_dict['own_recommender_type'].append(own_recommender)
#     result_dict['user_item_matrix_values'].append(ui_value)
    
#     for el in recs_type_list:
#         res = base_recommender.evalMetrics(metric_type='recall', df_result=data_val_matcher, 
#                         target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=N_PREDICT)
#         result_dict[el + '_recall'].append(res)

In [19]:
# result_df = pd.DataFrame(result_dict)
# result_df

In [20]:
# for col in result_df.columns.to_list()[4:]:
#     print(f'Best {col}:\n{result_df.loc[np.argmax(result_df[col]), :]}')
#     print('*' * 30)

# Init/train recommender

In [21]:
recommender = MainRecommender(data_train_matcher, weighting='tfidf',
                                 model_type='als', own_recommender_type='cosine')

### Recall@50 of matching

In [22]:
recs_type_list = ['own', 'rec', 'itm', 'usr']

In [23]:
TOPK_RECALL = 10

In [24]:
# for el in recs_type_list:
#     res = recommender.evalMetrics(metric_type='recall', df_result=data_val_matcher, 
#                     target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_RECALL)
#     print(f'{el} recall: {res}')

### Precision@5 of matching

In [25]:
TOPK_PRECISION = 5

In [26]:
# for el in recs_type_list:
#     res = recommender.evalMetrics(metric_type='precision', df_result=data_val_matcher, 
#                     target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION)
#     print(f'{el} precision: {res}')

## Второй этап отбора кандидатов
Создаю датафрейм из кандидатов, отобранных на первом этапе

In [27]:
df_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_candidates.columns = [USER_COL]

In [28]:
df_candidates['candidates'] = df_candidates[USER_COL].apply(
                              lambda x: recommender.get_own_recommendations(x, N=100))

In [29]:
df_items = df_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack(). \
                               reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [30]:
df_candidates = df_candidates.drop('candidates', axis=1).join(df_items)

In [31]:
df_candidates.head()

,user_id,item_id
0,2070,1029743
0,2070,913210
0,2070,1105426
0,2070,933067
0,2070,838186


In [32]:
# Check warm start
print_stats_data(df_candidates, 'candidates')

candidates
Shape: (215100, 2) Users: 2151 Items: 17612


In [33]:
df_train_ranker = df_candidates.merge(data_train_ranker, on=[USER_COL, ITEM_COL])
df_train_ranker.head()

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2070,913210,40764912478,605,1,3.59,311,-0.40,1900,87,0.0,0.0
1,2070,933067,40764912478,605,2,5.00,311,-2.98,1900,87,0.0,0.0
2,2070,838186,40764912478,605,1,3.99,311,0.00,1900,87,0.0,0.0
3,2070,1092937,40941467404,619,1,3.29,311,0.00,2015,89,0.0,0.0
4,2070,1092937,41160132372,630,1,3.29,311,0.00,14,91,0.0,0.0


Обучаю вторую модель

In [34]:
recommender_2 = MainRecommender(df_train_ranker, weighting='tfidf',
                                model_type='als', own_recommender_type='cosine')

In [35]:
for el in recs_type_list:
    res = recommender_2.evalMetrics(metric_type='recall', df_result=df_train_ranker, 
                    target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_RECALL,
                                   filter_items=False)
    print(f'{el} recall: {res}')

KeyError: 999999

In [ ]:
for el in recs_type_list:
    res = recommender_2.evalMetrics(metric_type='precision', df_result=df_train_ranker, 
                    target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION,
                                   filter_items=False)
    print(f'{el} precision: {res}')

# Ranking part

## Подготовка данных для трейна

In [ ]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [ ]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [ ]:
df_match_candidates.head(2)

In [ ]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [ ]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [ ]:
df_match_candidates.head(4)

### Check warm start

In [ ]:
print_stats_data(df_match_candidates, 'match_candidates')

### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [ ]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

In [ ]:
df_ranker_train.head()

#### Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей

In [ ]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [ ]:
df_ranker_train.target.value_counts()

In [ ]:
df_ranker_train.head(2)

## Подготавливаем фичи для обучения модели

In [ ]:
item_features.head(2)

In [ ]:
user_features.head(2)

In [ ]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

### Добавляю новые фичи в датасет

Заказ товара в последних 5 покупках в виде последовательности бит (категориальная).

In [ ]:
last_5_purchases_as_bit_array = ft.get_last_5_purchases_as_bit_array(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(last_5_purchases_as_bit_array, on=['item_id'], how='left')

#### user_id

In [ ]:
# Средняя сумма покупки 1 товара в каждой категории
mean_purchase_by_department= ft.get_mean_purchase_per_item_by_department(data_train_ranker, 
                                                                      item_features)
df_ranker_train = df_ranker_train.merge(mean_purchase_by_department[['user_id', 
                        'department', 'mean_purchase']], on=['user_id', 'department'], how='left')

# Кол-во покупок в каждой категории
num_purchases_by_department = ft.get_num_purchases_per_department(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(num_purchases_by_department[['user_id', 'department', 
                                        'num_purchases']], on=['user_id', 'department'], how='left')

# Доля покупок утром/днем/вечером
user_trans_df = ft.get_proportion_of_purchases_by_times_of_day(data_train_ranker)
df_ranker_train = df_ranker_train.merge(user_trans_df, on='user_id', how='left')

#### item_id

In [ ]:
# Кол-во покупок в неделю
week_purchases_df = ft.get_num_purchases_per_week(data_train_ranker)
df_ranker_train = df_ranker_train.merge(week_purchases_df[['item_id', 'n_purchases_per_week']], 
                                        on=['item_id'], how='left')

# Среднее кол-во покупок 1 товара в категории в неделю
mean_n_purchases_per_week = ft.get_mean_num_purchases_per_item_dept_week(data_train_ranker, 
                                                                      item_features)
df_ranker_train = df_ranker_train.merge(mean_n_purchases_per_week[['department', 
                                        'mean_n_purchases_per_week']], on=['department'], how='left')

# (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)
df_ranker_train['n_purchases_div_by_mean'] = \
    df_ranker_train.n_purchases_per_week / df_ranker_train.mean_n_purchases_per_week

# Цена
item_price_df = ft.get_price(data_train_ranker)
df_ranker_train = df_ranker_train.merge(item_price_df[['item_id', 'price']], 
                                        on=['item_id'], how='left')

# Цена / Средняя цена товара в категории
df_ranker_train.loc[(df_ranker_train.department == 'PRODUCE') &
                    (df_ranker_train.price == float('Inf')), 'price'] = 0
mean_price_by_department = ft.get_mean_price_by_department(df_ranker_train)
df_ranker_train = df_ranker_train.merge(mean_price_by_department[['department', 'mean_price']], 
                                        on=['department'], how='left')
df_ranker_train['price_div_by_mean_dept_price'] = df_ranker_train.price / df_ranker_train.mean_price
df_ranker_train.drop('mean_price', axis=1, inplace=True)

### user_id - item_id

In [ ]:
# (Кол-во покупок юзером конкретной категории в неделю) - (Среднее 
# кол-во покупок всеми юзерами конкретной категории в неделю)
n_purchases_sub_by_mean = ft.get_num_purchases_sub_by_mean(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(n_purchases_sub_by_mean[['user_id', 'department',
        'n_purchases_sub_by_mean']], on=['user_id', 'department'], how='left')

# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее 
# кол-во покупок всеми юзерами конкретной категории в неделю)
n_purchases_div_by_mean = ft.get_num_purchases_div_by_mean_all_users(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(n_purchases_div_by_mean[['user_id', 'department',
        'n_purchases_div_by_mean_all_users']], on=['user_id', 'department'], how='left')

# (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
sales_values_by_dept = ft.get_mean_sales_value_per_item_by_department(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(sales_values_by_dept[['department', 'mean_sale_sum_per_item']], 
                                        on=['department'], how='left')
df_ranker_train['mean_sale_sum_per_item_sub_price'] = \
    df_ranker_train.mean_sale_sum_per_item - df_ranker_train.price
df_ranker_train.drop('mean_sale_sum_per_item', axis=1, inplace=True)

Поведенческие фичи

In [ ]:
# Общая сумма покупок каждого товара
total_item_sales_value = ft.get_total_item_sales_value(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(total_item_sales_value, how='left', on='item_id')

# Общее количество по каждому товару
total_quantity_value = ft.get_total_quantity_value(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(total_quantity_value, how='left', on='item_id')

# Количество покупателей по каждому товару
item_freq = ft.get_item_freq(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_freq, how='left', on='item_id')

# Частота пользователей
user_freq = ft.get_user_freq(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_freq, how='left', on=USER_COL)

# Общее количество покупок по каждому пользователю
total_user_sales_value = ft.get_total_user_sales_value(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(total_user_sales_value, how='left', on='user_id')

# Среднее количество покупок товара в неделю
item_quantity_per_week = ft.get_item_quantity_per_week(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_quantity_per_week, how='left', on='item_id')

# Среднее количество купленного товара пользователем в неделю
user_quantity_per_week = ft.get_user_quantity_per_week(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_quantity_per_week, how='left', on='user_id')

# Среднее количество товара за 1 покупку
item_quantity_per_basket = ft.get_item_quantity_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_quantity_per_basket, how='left', on='item_id')

# Среднее количество товара у польователя за 1 покупку
user_quantity_per_basket = ft.get_user_quantity_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_quantity_per_basket, how='left', on='user_id')

# Средняя частота товара в карзине
item_freq_per_basket = ft.get_item_freq_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_freq_per_basket, how='left', on='item_id')

# Средняя частота пользователей купивших товар
user_freq_per_basket = ft.get_user_freq_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_freq_per_basket, how='left', on='user_id')

Факторы товаров из модели матричной факторизации

In [ ]:
# item_factors = pd.DataFrame(recommender.model.item_factors)
# item_factors.columns = [f'item_factor_{i}' for i in range(len(item_factors.columns))]
# item_ids = [recommender.id_to_itemid[itm_id] for itm_id in range(item_factors.shape[0])]
# item_factors = pd.concat([pd.DataFrame(item_ids), item_factors], axis=1)
# item_factors.rename(columns={0: 'item_id'}, inplace=True)
# item_factors.head()

In [ ]:
item_factors = recommender.get_item_factors()
df_ranker_train = df_ranker_train.merge(item_factors, on=['item_id'], how='left')

Факторы пользователей из модели матричной факторизации

In [ ]:
# user_factors = pd.DataFrame(recommender.model.user_factors)
# user_factors.columns = [f'user_factor_{i}' for i in range(len(user_factors.columns))]
# user_ids = [recommender.id_to_userid[usr_id] for usr_id in range(user_factors.shape[0])]
# user_factors = pd.concat([pd.DataFrame(user_ids), user_factors], axis=1)
# user_factors.rename(columns={0: 'user_id'}, inplace=True)
# user_factors.head()

In [ ]:
user_factors = recommender.get_user_factors()
df_ranker_train = df_ranker_train.merge(user_factors, on=['user_id'], how='left')

### Разбиваю на X и y

In [ ]:
df_ranker_train.head()

In [ ]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [ ]:
cat_feats = X_train.columns[2:16].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

## Обучение модели ранжирования

In [ ]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=500,
                     learning_rate=0.1,
                     categorical_column=cat_feats,
                     random_state=0)

In [ ]:
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [ ]:
df_ranker_predict = df_ranker_train.copy()

In [ ]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [ ]:
df_ranker_predict.head()

# Evaluation on test dataset

In [ ]:
# print('Matcher\'s precision:')
# for el in recs_type_list:
#     res = recommender.evalMetrics(metric_type='precision', df_result=data_val_matcher, 
#                     target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION)
#     print(f'{el} precision: {res}')

In [ ]:
# print('Ranker\'s precision:')
# for el in recs_type_list:
#     res = recommender.evalMetrics(metric_type='precision', df_result=data_val_ranker, 
#                     target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION)
#     print(f'{el} precision: {res}')

## Eval re-ranked matched result on test dataset

In [ ]:
print('Re-ranked precision:')
recommender.reranked_metrics(metric_type='precision', df_result=data_val_ranker, 
                                  df_predict=df_ranker_predict, target_col_name=USER_COL, 
                                  recommend_model_type='own', N_PREDICT=TOPK_PRECISION)

# Оценка на тесте для выполнения курсового проекта

In [ ]:
# df_test = pd.read_csv('./data/retail_test1.csv')
# df_test.head()

In [ ]:
# warm start
# df_test = df_test[df_test.user_id.isin(common_users)]
# print_stats_data(df_test,'df_test')

In [ ]:
# print('Test precision:')
# recommender.evalMetrics(metric_type='precision', df_result=df_test, 
#                 target_col_name=USER_COL, recommend_model_type='own', N_PREDICT=TOPK_PRECISION)

In [ ]:
# print('Test re-ranked precision:')
# recommender.reranked_metrics(metric_type='precision', df_result=df_test, 
#                                   df_predict=df_ranker_predict, target_col_name=USER_COL, 
#                                   recommend_model_type='own', N_PREDICT=TOPK_PRECISION)